In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from xgboost import XGBRanker

In [2]:
df = pd.read_csv("G:\My Drive\projects\data_science\working\dataset\student_profiles_updated.csv")
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\gayat\AppData\Local\Temp\ipykernel_5944\1063429385.py:1: SyntaxWarning: invalid escape sequence '\M'
  df = pd.read_csv("G:\My Drive\projects\data_science\working\dataset\student_profiles_updated.csv")


,Name,10th marks,12th course,12th marks,grad course,grad stream,grad marks,skills,languages known,internships taken,additional certifications,DOB,Category
0,YOVITHA,62.9,NaN,NaN,NaN,NaN,NaN,NaN,"Punjabi, English, Hindi",NaN,"Certificate in IFRS, Oracle Java Certification",2001-05-21,MBC
1,SUKUNNAN,93.4,Arts,85.0,BCom,Electrical Engineering,71.8,"Excel, Communication","Urdu, Hindi, Tamil",NaN,NaN,2004-08-18,BC
2,SHOORSEN,65.6,Arts,69.4,BTech,Mechanical Engineering,65.2,NaN,NaN,NaN,NaN,2003-06-30,BC
3,AADALALAGAN,76.2,Commerce,69.4,NaN,NaN,NaN,NaN,"Tamil, Gujarati",NaN,NaN,2001-07-25,BC
4,SKANDAPRASAD,86.2,Science,89.7,NaN,NaN,NaN,NaN,NaN,Finance Intern at Local Firm,NaN,2002-10-06,MBC


In [5]:
txt = r"""Internship Title:
Data Science Intern – Predictive Analytics

Requirements:

Must be a final-year B.Tech / B.Sc in Computer Science, IT, or related field

Knowledge of Python, Machine Learning, and Data Analysis

Familiarity with pandas, NumPy, and basic statistics

Understanding of data visualization tools (Matplotlib / Seaborn)

Bonus if you know SQL

Languages Preferred:

English (must)

Hindi or any regional language is a plus

Additional Notes:

Prior internships or certifications in Data Science will be considered an advantage

Location preference: Remote"""

skills = "Python, Machine Learning, Data Analysis, Pandas, Numpy, basic statistics, matplotlib, seaborn, SQL, Data Science"
mandatory_languages = "English"
accepted_languages = "Hindi"
needed_course = ["BTech","BSc", "BE"]
needed_stream = ["Computer Science", "IT", "AI-DS"]

In [6]:
df_filtered = df[df['grad course'].isin(needed_course) & df['grad stream'].isin(needed_stream)]
df_filtered = df_filtered[df_filtered['languages known'].str.contains(mandatory_languages, case=False, na=False)]

In [7]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
df_filtered["skills"] = df_filtered["skills"].fillna("")
candidate_embeddings = model.encode(df_filtered["skills"].tolist(), convert_to_tensor=True)
internship_embeddings = model.encode(txt, convert_to_tensor=True)
similarity_matrix = util.cos_sim(candidate_embeddings, internship_embeddings)
df_filtered["similarity_score"] = similarity_matrix.squeeze().tolist()
df_sorted = df_filtered.sort_values(by="similarity_score", ascending=False)
print(df_sorted[["Name", "skills", "similarity_score"]])

              Name                      skills  similarity_score
43851  CHANDRABHAN            Java, Python, ML          0.385909
5183      ABHIRATH      ML, Git, Data Analysis          0.384479
35871      SARJANA  Data Analysis, Writing, ML          0.367051
31713    THEEPIKAA   Excel, CAD, Data Analysis          0.359438
8614      BAHARUPA            Stata, Java, CAD          0.339511
...            ...                         ...               ...
4880       NAVNITA                                      0.030337
17585      AMEESHA                                      0.030337
11873       AMULYA             Public speaking         -0.001424
1388    SHREESANTH             Public speaking         -0.001424
39897     KAARIKAI             Public speaking         -0.001424

[190 rows x 3 columns]
